Efficiency Example
====

The first cell is just helper functions; the second cell is where everything happens

In [ ]:
import uproot
import phasespace
import numpy as np
import matplotlib.pyplot as plt
from hep_ml.reweight import GBReweighter

from typing import Tuple
from fourbody.param import helicity_param

In [ ]:
def _read_branches(tree, i: int) -> np.ndarray:
    """
    Read momentum and energy branches for the i'th pion
    
    """
    # Keep only the ones where D0 bkgcat == 0
    keep = tree["D0_BKGCAT"].array() == 0
    
    rv = np.row_stack([tree[f"Pion{i}_TRUEP_{x}"] for x in ("X", "Y", "Z", "E")])
    return rv[:, keep]

def _read_mc() -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Read that MC file you gave me
    
    Returns arrays: (px, py, pz, E)
    
    :returns: (4, N) shape numpy array for pi1
    :returns: (4, N) shape numpy array for pi2
    :returns: (4, N) shape numpy array for pi3
    :returns: (4, N) shape numpy array for pi4
    :returns: array of weights
    
    """
    with uproot.open("Total_DK_as_DK_MC_2017_After_PIC_Calib2_pols_merged_with_PID.root") as f:
        tree = f["DecayTree"]
        
        pi1, pi2, pi3, pi4 = (_read_branches(tree, i) for i in range(1, 5))
        
    return pi1, pi2, pi3, pi4


def _generate_flat() -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Generate flat (phsp) events according to D -> 4pi
    
    Returns arrays: (px, py, pz, E)
    
    :returns: (4, N) shape numpy array for pi1
    :returns: (4, N) shape numpy array for pi2
    :returns: (4, N) shape numpy array for pi3
    :returns: (4, N) shape numpy array for pi4
    """
    D0_MASS = 1869.62
    PION_MASS = 139.57018

    # The phasespace package returns weights
    # You could do an accept-reject here to keep only the right events but instead here I'm just going to keep the weights
    # and pass them to the reweighter
    weights, particles = phasespace.nbody_decay(D0_MASS,
                                                [PION_MASS, PION_MASS, PION_MASS, PION_MASS]
                                               ).generate(n_events=100000)
    
    # Doesn't matter which pion is + or - since they're all identical here
    pi1, pi2, pi3, pi4 = (particles[f"p_{i}"].numpy().T for i in range(4))
    
    return pi1, pi2, pi3, pi4, weights

def _plot_stuff(mc, flat, mc_wt, flat_wt):
    fig, ax = plt.subplots(3, 2, figsize=(12, 8))
    labels = (
        r"$M(\pi_1\pi_3) /MeV$",
        r"$M(\pi_2\pi_4) /MeV$",
        r"cos($\theta_{13}$)",
        r"cos($\theta_{24}$)",
        r"$\phi$")
    kw = {"histtype": "step", "density": True}
    for a, m, f, l in zip(ax.ravel()[:-1], mc.T, flat.T, labels):
        _, bins, _ = a.hist(m, **kw, label="MC", bins=50, alpha=0.5)
        a.hist(f, **kw, label="Flat", bins=bins, weights=flat_wt)
        a.hist(m, **kw, label="Reweighted", bins=bins, weights=mc_wt)
        a.set_xlabel(l)
    
    ax.ravel()[0].legend()
    fig.tight_layout()
    plt.show()

In [ ]:
# Read the right branches in the data file - we just care about kinematic information (momenta, energy)
mc_pi1, mc_pi2, mc_pi3, mc_pi4 = _read_mc()

# Generate "flat" (phase space distributed) events to compare the MC to
flat_pi1, flat_pi2, flat_pi3, flat_pi4, flat_weights = _generate_flat()

# Parameterise both into 5d point
# NB this function accepts (pi+ pi+ pi- pi-) (or pi- pi- pi+ pi+)
mc = helicity_param(mc_pi1, mc_pi3, mc_pi2, mc_pi4)

# You will need to add the same cuts to this flat data that you have used in the MC
# e.g. it looks like there might be a Ks veto?
flat = helicity_param(flat_pi1, flat_pi2, flat_pi3, flat_pi4)

# Train a reweighter
reweighter = GBReweighter()
reweighter.fit(target=flat, original=mc, target_weight=flat_weights)
efficiency_weights = reweighter.predict_weights(mc)

# Plot
_plot_stuff(mc, flat, efficiency_weights, flat_weights)